<a href="https://colab.research.google.com/github/mmartigani/RoadMapIA/blob/main/Fine_Tuning_chat_gpt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Fine-tuning de GPT 3 y ChatGPT para la invocación de funciones externas
Enunciado del caso práctico
En este caso práctico, se propone al alumno la realización de Fine-tuning sobre GPT-3.5-Turbo (ChatGPT)
 para mejorar la capacidad de detectar cuando debe invocarse una función externa.

Concretamente, se propone un escenario en el que una empresa quiere poner a
disposición de sus empleados un bot que les permita obtener información de sus
clientes de un aplicación corporativa denominada clientdb.

El bot utiliza el LLM GPT-3.5-turbo (ChatGPT) y debe ser capaz de
identificar cuando un empleado le solicita información de un cliente y
 extraer el nombre completo del mismo.


In [ ]:
User: Necesito información sobre el cliente zerocool

User: Busca en la base de datos de clientes a zerocool

User: Dame la información de zerocool consultando clientdb

User: Necesito las últimas transacciones de zerocool

User: Necesito todas las transacciones de un cliente
Assistant: Necesito que me proporciones el nombre del cliente
User: El nombre del cliente es zerocool

User: Necesito la información de un cliente
Assistant: Necesito que me proporciones el nombre del cliente
User: zerocool

User: Necesito información
Assistant: Si necesitas información sobre un cliente, necesito que me proporciones su nombre
User: Su nombre es zerocool

User: Necesito información sobre Santiago
Assistant: Necesito que me proporciones el nombre completo del cliente
User: zerocool

User: Dame la información de Hernández Ramos
Assistant: Necesito que me proporciones el nombre completo del cliente
User: Su nombre completo es zerocool

User: Realiza una consulta a clientdb
Assistant: Necesito que me proporciones el nombre completo del cliente
User: El nombre es zerocool

In [ ]:
!pip install openai==0.28

In [ ]:
import openai

with open("/content/drive/MyDrive/api-keys/secret-key.txt") as f:
  openai.api_key = f.readline()

In [ ]:
{
    "messages": [
        {"role": "user", "content": "Necesito informacion sobre el cliente Santiago Hernandez Ramos"},
        {"role": "assistant", "function_call": {"name": "clientdb", "arguments": "{\"usuario\": \"Santiago Hernandez Ramos\"}"}}
    ],
    "functions": [{
        "name": "clientdb",
        "description": "Proporciona la informacion de un cliente de la empresa",
        "parameters": {
            "type": "object",
            "properties": {
                "usuario": {"type": "string", "description": "El nombre completo del usuario del que se va a obtener la informacion, ej. Santiago Hernandez Ramos"}
            },
            "required": ["usuario"]
        }
    }]
}

In [ ]:
# Lectura del conjunto de datos
openai.File.create(
  file=open("/content/drive/MyDrive/datasets/functions_dataset.jsonl", "rb"),
  purpose='fine-tune'
)

In [ ]:
# Definicion de los hiperparametros
hyperparameters = {"n_epochs":3}

In [ ]:
# Fine-tuning del modelo
openai.FineTuningJob.create(
    training_file="file-1ca0m3Lq1HNA0gXfzzymRUjq", # Debe indicarse el id obtenido en la seccion anterior
    model="gpt-3.5-turbo",
    hyperparameters=hyperparameters)

In [ ]:
FT_JOB_ID = "ftjob-IlaEg77SNsbAWFX6jIFKs6im"

In [ ]:
def clientdb(usuario):
  return f"""
-----------------------------------
    INFORMACIÓN DEL CLIENTE
-----------------------------------

Nombre: {usuario}

Transacciones:
1) Fecha: 01/01/2023 - Monto: $200.00 - Concepto: Compra material de oficina
2) Fecha: 05/01/2023 - Monto: -$50.00 - Concepto: Devolución de producto
3) Fecha: 10/01/2023 - Monto: $120.00 - Concepto: Servicios contratados

Teléfono: (555) 123-4567

Dirección: Calle Ficticia 123, Ciudad Imaginaria, 00000

Email: {'.'.join(usuario.lower().split(' '))}@example.com

Notas:
- Prefiere ser contactado en horario de tarde.
- Ha expresado interés en nuevos productos relacionados con software empresarial.

Última interacción:
Fecha: 15/01/2023
Detalle: Se reunió con el representante de ventas para discutir nuevas ofertas.

-----------------------------------
"""

In [ ]:
print(clientdb("zerocool"))

In [ ]:
functions = [
        {
            "name": "clientdb",
            "description": "Proporciona la informacion de un cliente de la empresa.",
            "parameters": {
                "type": "object",
                "properties": {
                    "usuario": {
                        "type": "string",
                        "description": "El nombre completo del usuario del que se va a obtener la informacion, ej. Santiago Hernandez Ramos",
                    }
                },
                "required": ["usuario"],
            },
        }
    ]

In [ ]:
def obtener_completion(mensajes, model="ft:gpt-3.5-turbo-0613:personal::8CsvktiD"):
  respuesta = openai.ChatCompletion.create(
      model=model,
      messages=mensajes,
      functions=functions, # Proporciono las funciones definidas previamente
      temperature=0, # Este hiperparámetro controla la aleatoriedad del modelo
  )
  return respuesta.choices[0].message # Retornamos el mensaje

In [ ]:
#/content/functions_dataset.jsonl

In [ ]:
import json

def collect_info(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response_message = obtener_completion(context)

    # Comprobamos si GPT quiere invocar una funcion
    if response_message.get("function_call"):
        # Invocamos la funcion
        available_functions = {
            "clientdb": clientdb,
        }  # Podríamos tener más de una función

        # Obtenemos la funcion que quiere invocar GPT
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        # Obtenemos los argumentos de la funcion proporcionados por GPT
        function_args = json.loads(response_message["function_call"]["arguments"])
        # Invocamos la funcion
        function_response = function_to_call(usuario=function_args.get("usuario"))

        # Enviamos la respuesta de la función a GPT
        context.append(response_message)  # Respuesta del assistant
        context.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # Contenido de la función

        response_message = obtener_completion(context)

    context.append(response_message)
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response_message['content'], width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

In [ ]:
def end_chat(event):
    panels.append(pn.pane.Alert("Chat terminado por el usuario.", alert_type='success'))
    context.append({'role': 'system', 'content':"Despídete del usuario de manera amable y amigable."})
    response_message = obtener_completion(context)
    context.append(response_message)
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response_message['content'], width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = []

context = [ {'role':'system', 'content':
"""
Eres un asistente virtual para gestionar y procesar información de clientes en \
una empresa. Interactúa amablemente con el usuario y solicítale el nombre completo \
de un cliente para comenzar a trabajar.
"""} ]


inp = pn.widgets.TextInput(value="Hola", placeholder='Introduce texto aqui...')
button_conversation = pn.widgets.Button(name="Chat!")
button_end_chat = pn.widgets.Button(name="Terminar Chat")

button_end_chat.on_click(end_chat)

interactive_conversation = pn.bind(collect_info, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation, button_end_chat),
    pn.panel(interactive_conversation, loading_indicator=True, sizing_mode="stretch_both"),
)

dashboard